<a href="https://colab.research.google.com/github/amitvashista/nifty_50/blob/stage1/Deep_Q_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
df=pd.read_csv("/content/NiftyOptionCE_2025-02-24.csv")
df1=pd.read_csv("/content/NiftyOptionCE_2025-02-25.csv")
df2=pd.read_csv("/content/NiftyOptionCE_2025-02-27.csv")

In [60]:
df=pd.concat([df,df1,df2])

In [61]:
df.columns

Index(['Strike Price', 'expiryDate', 'openInterest', 'changeinOpenInterest',
       'pchangeinOpenInterest', 'totalTradedVolume', 'impliedVolatility',
       'lastPrice', 'change', 'pChange', 'totalBuyQuantity',
       'totalSellQuantity', 'bidQty', 'bidprice', 'askQty', 'askPrice',
       'underlyingValue', 'date', 'Time'],
      dtype='object')

In [62]:
# Standardize column names by removing spaces and making lowercase
df.columns = df.columns.str.replace(" ", "").str.lower()

# Convert 'date' and 'time' into a proper datetime column with corrected format
df["datetime"] = pd.to_datetime(df["date"] + " " + df["time"], format="%Y-%m-%d %H-%M-%S")


In [63]:
df.head()

,strikeprice,expirydate,openinterest,changeinopeninterest,pchangeinopeninterest,totaltradedvolume,impliedvolatility,lastprice,change,pchange,totalbuyquantity,totalsellquantity,bidqty,bidprice,askqty,askprice,underlyingvalue,date,time,datetime
0,17000,24-Dec-2025,9.333333,0.0,0.000000,0,0.00,0.00,0.00,0.000000,150,225,75,5802.65,75,6998.90,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45
1,18000,27-Mar-2025,6188.000000,79.0,1.293174,363,26.89,4701.15,-205.65,-4.191123,8550,9150,75,4699.60,75,4701.75,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45
2,18000,24-Dec-2025,125.000000,0.0,0.000000,0,0.00,0.00,0.00,0.000000,300,450,75,5350.00,75,5614.05,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45
3,18000,25-Jun-2026,1.333333,0.0,0.000000,0,0.00,0.00,0.00,0.000000,0,0,0,0.00,0,0.00,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45
4,19000,27-Mar-2025,818.333333,168.0,25.832906,205,0.00,3710.20,-209.80,-5.352041,9675,10575,75,3710.55,75,3721.20,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45


In [64]:
# prompt: convert expirydate column in to datetime column

import pandas as pd
# Convert 'expirydate' to datetime
df["expirydate"] = pd.to_datetime(df["expirydate"], format="%d-%b-%Y")
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')


In [65]:
df['remaining']=df['expirydate']-df['date']
df['remaining'] = df['remaining'].dt.days

In [66]:
df

,strikeprice,expirydate,openinterest,changeinopeninterest,pchangeinopeninterest,totaltradedvolume,impliedvolatility,lastprice,change,pchange,...,totalsellquantity,bidqty,bidprice,askqty,askprice,underlyingvalue,date,time,datetime,remaining
0,17000,2025-12-24,9.333333,0.0,0.000000,0,0.00,0.00,0.00,0.000000,...,225,75,5802.65,75,6998.90,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45,303
1,18000,2025-03-27,6188.000000,79.0,1.293174,363,26.89,4701.15,-205.65,-4.191123,...,9150,75,4699.60,75,4701.75,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45,31
2,18000,2025-12-24,125.000000,0.0,0.000000,0,0.00,0.00,0.00,0.000000,...,450,75,5350.00,75,5614.05,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45,303
3,18000,2026-06-25,1.333333,0.0,0.000000,0,0.00,0.00,0.00,0.000000,...,0,0,0.00,0,0.00,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45,486
4,19000,2025-03-27,818.333333,168.0,25.832906,205,0.00,3710.20,-209.80,-5.352041,...,10575,75,3710.55,75,3721.20,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236581,25250,2025-03-06,86.000000,36.0,72.000000,587,30.21,1.20,-0.45,-27.272727,...,45675,9600,1.00,375,1.15,22548.1,2025-02-27,14-42-51,2025-02-27 14:42:51,7
236582,25350,2025-03-27,103.000000,-2.0,-1.904762,73,17.52,4.80,-0.40,-7.692308,...,8325,225,4.20,225,5.55,22548.1,2025-02-27,14-42-51,2025-02-27 14:42:51,28
236583,25400,2025-03-27,273.000000,7.0,2.631579,159,17.72,4.30,-0.75,-14.851485,...,12225,75,4.30,450,4.50,22548.1,2025-02-27,14-42-51,2025-02-27 14:42:51,28
236584,25450,2025-02-27,545.000000,256.0,88.581315,1246,71.55,0.25,-0.30,-54.545455,...,121275,394125,0.15,31200,0.25,22548.1,2025-02-27,14-42-51,2025-02-27 14:42:51,0


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 484002 entries, 0 to 236585
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   strikeprice            484002 non-null  int64         
 1   expirydate             484002 non-null  datetime64[ns]
 2   openinterest           484002 non-null  float64       
 3   changeinopeninterest   484002 non-null  float64       
 4   pchangeinopeninterest  484002 non-null  float64       
 5   totaltradedvolume      484002 non-null  int64         
 6   impliedvolatility      484002 non-null  float64       
 7   lastprice              484002 non-null  float64       
 8   change                 484002 non-null  float64       
 9   pchange                484002 non-null  float64       
 10  totalbuyquantity       484002 non-null  int64         
 11  totalsellquantity      484002 non-null  int64         
 12  bidqty                 484002 non-null  int64    

In [68]:



# Select relevant columns with updated names
# df = df[["strikeprice", "expirydate", "datetime", "lastprice"]]

# Define function to resample into 5-minute candlestick format
def resample_candlestick(group):

    # group = group.set_index("datetime").sort_index()
    # return group.rolling("5min", closed="right").agg(
    #     {"lastprice": [lambda x: x.iloc[0], "max", "min", lambda x: x.iloc[-1]]}
    # ).dropna()

    resampled = group.resample("5min", closed="right", on="datetime").agg(
        {
            "lastprice": ["first", "max", "min", "last"],  # Open, High, Low, Close
            "totaltradedvolume": "sum",  # Example: Summing volume
            "openinterest": "last",  # Example: Taking the last value of Open Interest
            "impliedvolatility":["first", "max", "min", "last"],
            'remaining': 'last'
        }
    ).dropna()
    resampled.index = resampled.index.shift(5, freq="min")
    return resampled
print(df.columns)
# Apply groupby and resampling
candlestick_df = df.groupby(["strikeprice", "expirydate"]).apply(resample_candlestick)

# Reset index and rename columns
candlestick_df.columns = ["Open", "High", "Low", "Close", "Total_Volume", "OI_Last","VI_Open", "VI_High", "VI_Low", "VI_Close",'remaining']
candlestick_df.reset_index(inplace=True)

# Display the processed dataset
candlestick_df.head()


Index(['strikeprice', 'expirydate', 'openinterest', 'changeinopeninterest',
       'pchangeinopeninterest', 'totaltradedvolume', 'impliedvolatility',
       'lastprice', 'change', 'pchange', 'totalbuyquantity',
       'totalsellquantity', 'bidqty', 'bidprice', 'askqty', 'askprice',
       'underlyingvalue', 'date', 'time', 'datetime', 'remaining'],
      dtype='object')


<ipython-input-68-ce9d77057ef7>:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  candlestick_df = df.groupby(["strikeprice", "expirydate"]).apply(resample_candlestick)


,strikeprice,expirydate,datetime,Open,High,Low,Close,Total_Volume,OI_Last,VI_Open,VI_High,VI_Low,VI_Close,remaining
0,17000,2025-06-26,2025-02-27 09:15:00,6850.0,6850.0,6850.0,6850.0,0,0.000000,0.0,0.0,0.0,0.0,119.0
1,17000,2025-12-24,2025-02-24 15:20:00,0.0,0.0,0.0,0.0,0,9.333333,0.0,0.0,0.0,0.0,303.0
2,17000,2025-12-24,2025-02-24 15:25:00,0.0,0.0,0.0,0.0,0,9.333333,0.0,0.0,0.0,0.0,303.0
3,17000,2025-12-24,2025-02-24 15:30:00,0.0,0.0,0.0,0.0,0,9.333333,0.0,0.0,0.0,0.0,303.0
4,17000,2025-12-24,2025-02-24 15:35:00,0.0,0.0,0.0,0.0,0,9.333333,0.0,0.0,0.0,0.0,303.0


In [69]:
candlestick_df[(candlestick_df['strikeprice']==22700) & (candlestick_df['expirydate']=='27-Feb-2025')]

,strikeprice,expirydate,datetime,Open,High,Low,Close,Total_Volume,OI_Last,VI_Open,VI_High,VI_Low,VI_Close,remaining
31319,22700,2025-02-27,2025-02-24 15:20:00,61.35,61.40,60.55,60.55,6892140,140372.0,13.34,13.36,13.34,13.34,3.0
31320,22700,2025-02-27,2025-02-24 15:25:00,61.05,61.05,58.95,59.55,15655738,135033.0,13.34,14.10,13.34,13.96,3.0
31321,22700,2025-02-27,2025-02-24 15:30:00,60.00,60.00,57.75,57.75,10560096,133523.0,14.10,14.10,13.96,14.10,3.0
31322,22700,2025-02-27,2025-02-24 15:35:00,57.30,57.30,56.60,56.60,7117732,131184.0,14.10,14.10,13.65,13.96,3.0
31323,22700,2025-02-27,2025-02-25 09:15:00,56.60,56.60,56.60,56.60,3562786,125217.0,13.30,13.65,13.30,13.65,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31452,22700,2025-02-27,2025-02-27 14:25:00,0.80,0.80,0.75,0.80,11722926,136991.0,6.70,6.70,6.35,6.35,0.0
31453,22700,2025-02-27,2025-02-27 14:30:00,0.75,0.80,0.50,0.55,19096082,134907.0,6.35,6.63,6.35,6.35,0.0
31454,22700,2025-02-27,2025-02-27 14:35:00,0.55,0.55,0.50,0.55,14751714,133814.0,6.35,6.63,6.11,6.11,0.0
31455,22700,2025-02-27,2025-02-27 14:40:00,0.45,0.45,0.40,0.45,19929452,127098.0,6.11,6.11,5.91,5.91,0.0


In [70]:
# prompt: add 4*10 column in df dataset which have 'VI_Open'	'VI_High'	'VI_Low'	'VI_Close' column of last 10 datapoint by groupby [strikeprice	expirydate] and sort by datetime

# Calculate rolling statistics for 'VI_Open', 'VI_High', 'VI_Low', 'VI_Close'
def rolling_stats(group):
    group = group.sort_values("datetime")
    for i in range(1, 11):  # Creating 10 lagged values
        group[f"VI_Open_{i}"] = group["VI_Open"].shift(i)
        group[f"VI_High_{i}"] = group["VI_High"].shift(i)
        group[f"VI_Low_{i}"] = group["VI_Low"].shift(i)
        group[f"VI_Close_{i}"] = group["VI_Close"].shift(i)
        group[f"Open_{i}"] = group["Open"].shift(i)
        group[f"High_{i}"] = group["High"].shift(i)
        group[f"Low_{i}"] = group["Low"].shift(i)
        group[f"Close_{i}"] = group["Close"].shift(i)
    return group

candlestick_df = candlestick_df.groupby(["strikeprice", "expirydate"]).apply(rolling_stats).reset_index(drop=True)
candlestick_df.head()


<ipython-input-70-1a3ff8ef7e5d>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  candlestick_df = candlestick_df.groupby(["strikeprice", "expirydate"]).apply(rolling_stats).reset_index(drop=True)


,strikeprice,expirydate,datetime,Open,High,Low,Close,Total_Volume,OI_Last,VI_Open,...,Low_9,Close_9,VI_Open_10,VI_High_10,VI_Low_10,VI_Close_10,Open_10,High_10,Low_10,Close_10
0,17000,2025-06-26,2025-02-27 09:15:00,6850.0,6850.0,6850.0,6850.0,0,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17000,2025-12-24,2025-02-24 15:20:00,0.0,0.0,0.0,0.0,0,9.333333,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17000,2025-12-24,2025-02-24 15:25:00,0.0,0.0,0.0,0.0,0,9.333333,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17000,2025-12-24,2025-02-24 15:30:00,0.0,0.0,0.0,0.0,0,9.333333,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17000,2025-12-24,2025-02-24 15:35:00,0.0,0.0,0.0,0.0,0,9.333333,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
a=candlestick_df[(candlestick_df['strikeprice']==22700) & (candlestick_df['expirydate']=='27-Feb-2025')]

In [72]:
a.to_csv('test.csv')

In [73]:
candlestick_df.shape

(92573, 94)

In [58]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque

# Define DQN Network
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Define Environment
class ProductEnv:
    def __init__(self):
        self.state_size = 3  # Example: [market demand, product age, price trend]
        self.action_size = 3  # Actions: [0: Do Nothing, 1: Launch, 2: Retrieve]
        self.state = np.random.rand(self.state_size)
        self.time_step = 0

    def step(self, action):
        self.time_step += 1
        reward = self.get_reward(action)
        self.state = np.random.rand(self.state_size)  # Simulate new market conditions
        done = self.time_step > 100  # End after 100 time steps
        return self.state, reward, done

    def get_reward(self, action):
        if action == 1:  # Launch
            return random.uniform(-10, 10)  # Profit or loss
        elif action == 2:  # Retrieve
            return random.uniform(0, 15)  # Profit from retrieval
        return 0  # Do nothing

    def reset(self):
        self.state = np.random.rand(self.state_size)
        self.time_step = 0
        return self.state

# Define Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = DQN(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            action_values = self.model(state)
        return torch.argmax(action_values).item()

    def train(self, batch_size=32):
        if len(self.memory) < batch_size:
            return
        batch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state in batch:
            target = reward + self.gamma * torch.max(self.model(torch.FloatTensor(next_state).unsqueeze(0))).item()
            predicted = self.model(torch.FloatTensor(state).unsqueeze(0))[0, action]
            loss = F.mse_loss(predicted, torch.tensor(target))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

    def remember(self, state, action, reward, next_state):
        self.memory.append((state, action, reward, next_state))

    def update_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

# Train the agent
env = ProductEnv()
agent = DQNAgent(env.state_size, env.action_size)

episodes = 1000
for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    while True:
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        agent.remember(state, action, reward, next_state)
        state = next_state
        total_reward += reward
        agent.train()
        if done:
            agent.update_epsilon()
            break
    print(f"Episode {episode + 1}, Total Reward: {total_reward:.2f}, Epsilon: {agent.epsilon:.3f}")


KeyboardInterrupt: 

In [ ]:
import numpy as np
import random
import pandas as pd

In [ ]:
a=pd.DataFrame([
            {
                'strike_price': random.randint(100, 500),
                'expiry':  random.randint(10, 30),
                'type': random.choice(['CALL', 'PUT']),
                'premium': random.uniform(5, 50),
                'candles': [  # Past OHLC for multiple timeframes
                    [random.uniform(100, 200) for _ in range(4)] for _ in range(10)
                ]
            }
            for _ in range(4)
        ])

In [ ]:
a

In [ ]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque

# Define the DQN Model
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Define Environment
class OptionsTradingEnv:
    def __init__(self, num_contracts=5, lookback=10):
        self.num_contracts = num_contracts  # Number of contracts to track
        self.lookback = lookback  # Candlestick history length
        self.state_size = num_contracts * (lookback * 4 + 4)  # OHLC + contract details
        self.action_size = num_contracts * 3  # (Neutral, Execute, Close)
        self.time_step = 0
        self.contracts = self.generate_contracts()
        self.open_positions = {}  # Track open contracts (id -> entry price)


    def generate_contracts(self):
        """ Generate contract data with candlesticks """

        return [
            {
                'strike_price': random.randint(100, 500),
                'expiry': self.time_step + random.randint(10, 30),
                'type': random.choice(['CALL', 'PUT']),
                'premium': random.uniform(5, 50),
                'candles': [  # Past OHLC for multiple timeframes
                    [random.uniform(100, 200) for _ in range(4)] for _ in range(self.lookback)
                ]
            }
            for _ in range(self.num_contracts)
        ]

    def step(self, action):
        """ Apply action and update state """
        self.time_step += 1
        reward = 0

        contract_id = action // 3  # Identify contract (0 to num_contracts - 1)
        action_type = action % 3   # (0 = Neutral, 1 = Execute, 2 = Close)

        if action_type == 1 and contract_id not in self.open_positions:  # Execute contract
            self.open_positions[contract_id] = self.contracts[contract_id]['premium']
            reward = -self.contracts[contract_id]['premium']  # Cost of buying the option

        elif action_type == 2 and contract_id in self.open_positions:  # Close contract
            exit_price = self.contracts[contract_id]['candles'][-1][3]  # Closing price
            entry_price = self.open_positions.pop(contract_id)
            reward = (exit_price - entry_price) * (1 if self.contracts[contract_id]['type'] == 'CALL' else -1)

        # Remove expired contracts
        self.open_positions = {k: v for k, v in self.open_positions.items() if self.contracts[k]['expiry'] > self.time_step}

        # Update state
        self.contracts = self.generate_contracts()
        state = self.get_state()
        done = self.time_step > 100  # End after 100 steps
        return state, reward, done

    def get_state(self):
        """ Flatten contract attributes into a single state vector """
        state = []
        for contract in self.contracts:
            for candle in contract['candles']:
                state.extend(candle)  # OHLC data
            state.extend([contract['strike_price'], contract['expiry'], int(contract['type'] == 'CALL'), contract['premium']])
        return np.array(state)

    def reset(self):
        self.time_step = 0
        self.open_positions = {}
        self.contracts = self.generate_contracts()
        return self.get_state()

# Define DQN Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = DQN(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def act(self, state):
        """ Select action based on epsilon-greedy strategy """
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0)                           # converts the NumPy array or list state into a PyTorch tensor of type float32
        with torch.no_grad():                                                   # It disables gradient computation in PyTorch. reducing memory usage and speeding up computations.
            action_values = self.model(state)                                   #passing state as vector in nn and get output
        return torch.argmax(action_values).item()                               #returns the index of the maximum value along a given dimension

    def train(self, batch_size=32):
        """ Train model using replay memory """
        if len(self.memory) < batch_size:
            return
        batch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state in batch:
            # not clear  ( calculate reward base on next state and update reward. but we don't have next state.)
            target = reward + self.gamma * torch.max(self.model(torch.FloatTensor(next_state).unsqueeze(0))).item()

            # not clear
            predicted = self.model(torch.FloatTensor(state).unsqueeze(0))[0, action]

            #calculate loss
            loss = F.mse_loss(predicted, torch.tensor(target))

            # update the weight after back propogation
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

    def remember(self, state, action, reward, next_state):
        """ Store experience in memory """
        self.memory.append((state, action, reward, next_state))

    def update_epsilon(self):
        """ Decay exploration rate """
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

# Train the agent
env = OptionsTradingEnv(num_contracts=5)
agent = DQNAgent(env.state_size, env.action_size)

episodes = 1000
for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    while True:
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        agent.remember(state, action, reward, next_state)
        state = next_state
        total_reward += reward
        agent.train()
        if done:
            agent.update_epsilon()
            break
    print(f"Episode {episode + 1}, Total Reward: {total_reward:.2f}, Epsilon: {agent.epsilon:.3f}")


In [ ]:
# Save only the model parameters (recommended)
torch.save(agent.model.state_dict(), "dqn_model.pth")

# Save the entire model (optional, but not recommended for versioning)
torch.save(agent.model, "dqn_model_full.pth")
